### An attempt to integrate PredPreyGrassEnv into ray rllib 2.9.3

Sourcsces of conversion problems:
#https://discuss.ray.io/t/multiagents-type-actions-observation-space-defined-in-environement/5120/3
#https://discuss.ray.io/t/observation-space-not-provided-in-policyspec/6501/10
#https://colab.research.google.com/drive/19aZ_bUFJwJuPT4dRzVO2cTn8Ey0G7CoA?usp=sharing#scrollTo=7GjQY33DTz0H
#https://github.com/ray-project/ray/blob/master/rllib/env/env_context.py

The relation between the observation space and an observation is that the observation is a sample from the observation space. [chatgpt]. The question remains how to check this.
https://stackoverflow.com/questions/76289764/understanding-action-observation-spaces-in-gym-for-custom-environments-and-age

TODO: 
-observe function adjustment
-swapaxes
-get rid of masks and observation range for observation spaces?




In [1]:
from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.utils.pre_checks.env import  check_env
import time
check_env(PredPreyGrassEnv())


pygame 2.3.0 (SDL 2.24.2, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
3
get_agent_ids {'predator_0', 'prey_4', 'predator_3', 'prey_10', 'prey_5', 'predator_2', 'prey_8', 'prey_11', 'prey_6', 'prey_9', 'predator_1', 'prey_7'}
4
get_agent_ids {'predator_0', 'prey_4', 'predator_3', 'prey_10', 'prey_5', 'predator_2', 'prey_8', 'prey_11', 'prey_6', 'prey_9', 'predator_1', 'prey_7'}


ValueError: Traceback (most recent call last):
  File "/home/doesburg/Dropbox/03_marl_code/PredPreyGrass/.conda/lib/python3.11/site-packages/ray/rllib/utils/pre_checks/env.py", line 307, in check_multiagent_environments
    obs_and_infos = env.reset(seed=42, options={})
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/doesburg/Dropbox/03_marl_code/PredPreyGrass/rllib/predpreygrass/environments/predpreygrass_env.py", line 294, in reset
    self.agent_observation_dict[agent_name] = self.observe(agent_name)
                                              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/doesburg/Dropbox/03_marl_code/PredPreyGrass/rllib/predpreygrass/environments/predpreygrass_env.py", line 535, in observe
    observation[i][0:max,max-1-j] = 0
    ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^
IndexError: index 6 is out of bounds for axis 1 with size 4

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/doesburg/Dropbox

In [ ]:
check_env(PredPreyGrassEnv())

In [ ]:

config = (
    PPOConfig()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]})
    .environment(env=PredPreyGrassEnv,disable_env_checking=False)
    .multi_agent(
        policies=["policy1", "policy2"],
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "policy1" if agent_id == "agent1" else "policy2"
    )
)
algo = config.build()

env = PredPreyGrassEnv(config={"render": True})
obs, _ = env.reset()
terminateds = {"__all__" : False}
truncateds = {"__all__" : False}
    
while not terminateds["__all__"] or not truncateds["__all__"]:

    action1 = algo.compute_single_action(obs["agent1"], policy_id="policy1")
    action2 = algo.compute_single_action(obs["agent2"], policy_id="policy2")

    obs, rewards, dones, _, infos = env.step({"agent1": action1, "agent2": action2})

    env.render()
    time.sleep(0.5)
    
algo.stop()

In [ ]:
#registering the environment
from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
import time

from ray.tune.registry import register_env

def env_creator(env_config):
    return PredPreyGrassEnv(env_config=env_config)  # return an env instance

register_env("pred_prey_grass", env_creator)

config = (
    PPOConfig()
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0, log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]})
    .environment(env="pred_prey_grass", disable_env_checking=False)
    .multi_agent(
        policies=["policy1", "policy2"],
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "policy1" if agent_id == "agent1" else "policy2"
    )
)

algo = config.build()

env = PredPreyGrassEnv(env_config={"render": True})
obs, _ = env.reset()
dones = {"__all__" : False}
    
while not dones["__all__"]:

    action1 = algo.compute_single_action(obs["agent1"], policy_id="policy1")
    action2 = algo.compute_single_action(obs["agent2"], policy_id="policy2")

    obs, rewards, dones, _, infos = env.step({"agent1": action1, "agent2": action2})

    env.render()
    time.sleep(0.5)
    
algo.stop()

In [ ]:
#define policies in dict with PolicySpec
from environments.predpreygrass_env import PredPreyGrassEnv
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

import time

def env_creator(env_config):
    return PredPreyGrassEnv(env_config=env_config)  # return an env instance

register_env("pred_prey_grass", env_creator)

policies = { 
    "policy1": PolicySpec(None,observation_space=PredPreyGrassEnv.observation_space, action_space=PredPreyGrassEnv.action_space),
    "policy2": PolicySpec(None,observation_space=PredPreyGrassEnv.observation_space, action_space=PredPreyGrassEnv.action_space)
}



config = (
    PPOConfig()
    .environment(env="pred_prey_grass", disable_env_checking=False)
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]})
    .multi_agent(
        policies=policies,
        policy_mapping_fn=lambda agent_id, episode, worker, **kwargs: "policy1" if agent_id.starts_with == "agent1" else "policy2"
    )
)

algo = config.build()
"""
config_env = {"render": True}


env = PredPreyGrassEnv(config=config_env)
obs, _ = env.reset()
dones = {"__all__" : False}
"""
    
"""
while not dones["__all__"]:

    action1 = algo.compute_single_action(obs["agent1"], policy_id="policy1")
    action2 = algo.compute_single_action(obs["agent2"], policy_id="policy2")

    obs, rewards, dones, _, infos = env.step({"agent1": action1, "agent2": action2})

    env.render()
    time.sleep(0.5)
    
algo.stop()
"""



In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment


from multi_agent_env_ray_2_9_3 import MultiAgentArena
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env

import time

MAX_WIDTH = 100
MAX_HEIGHT = 100
                                                   
def env_creator(env_config):
    print(env_config)
    return MultiAgentArena(config=env_config, width=MAX_WIDTH, height=MAX_HEIGHT)  # return an env instance
register_env("multi_agent_arena", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == "agent1":
        return "policy1"
    else:
        return "policy2"



config = (
    PPOConfig()
    .environment(env="multi_agent_arena")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]})
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)

algo = config.build()

config_env = {"render": True}


env = MultiAgentArena(config={"render": True}, width=10, height=10)
obs, _ = env.reset()
dones = {"__all__" : False}

    

while not dones["__all__"]:

    action1 = algo.compute_single_action(obs["agent1"], policy_id="policy1")
    action2 = algo.compute_single_action(obs["agent2"], policy_id="policy2")

    obs, rewards, dones, _, infos = env.step({"agent1": action1, "agent2": action2})

    env.render()
    time.sleep(0.5)
    
algo.stop()



In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
#train the environment with the algo

from multi_agent_env_ray_2_9_3 import MultiAgentArena
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import time

MAX_WIDTH = 10
MAX_HEIGHT = 10
                                                   
def env_creator(env_config):
    print(env_config)
    return MultiAgentArena(config=env_config, width=MAX_WIDTH, height=MAX_HEIGHT)  # return an env instance
register_env("multi_agent_arena", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == "agent1":
        return "policy1"
    else:
        return "policy2"


algo = (
    PPOConfig()
    .environment(env="multi_agent_arena")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]})
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
    .build()
)

for i in range(10):
    result = algo.train()
    print(pretty_print(result))

    if i % 5 == 0:
        checkpoint_dir = algo.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")


config_env = {"render": True}


env = MultiAgentArena(config={"render": True}, width=10, height=10)
obs, _ = env.reset()
dones = {"__all__" : False}
    

while not dones["__all__"]:

    action1 = algo.compute_single_action(obs["agent1"], policy_id="policy1")
    action2 = algo.compute_single_action(obs["agent2"], policy_id="policy2")

    obs, rewards, dones, _, infos = env.step({"agent1": action1, "agent2": action2})

    env.render()
    time.sleep(0.5)
    
algo.stop()




In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
#train the environment with the algo
#use tune for a gridsearch

import ray
from ray import train, tune

from multi_agent_env_ray_2_9_3 import MultiAgentArena
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import time

MAX_WIDTH = 10
MAX_HEIGHT = 10

ray.init(ignore_reinit_error=True)
                                                   
def env_creator(env_config):
    print(env_config)
    return MultiAgentArena(config=env_config, width=MAX_WIDTH, height=MAX_HEIGHT)  # return an env instance
register_env("multi_agent_arena", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == "agent1":
        return "policy1"
    else:
        return "policy2"


config = (
    PPOConfig()
    .environment(env="multi_agent_arena")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]},lr=tune.grid_search([0.01, 0.001, 0.0001]))
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


tuner = tune.Tuner(
    "PPO",
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 15},
    ),
    param_space=config,
)

tuner.fit()



In [ ]:
#define policies in dict with PolicySpec
#add width and height parameters to the environment
#train the environment with the algo
#use tune for a gridsearch
#retrieving the checkpoint(s) of the trained agent

import ray
from ray import train, tune

from multi_agent_env_ray_2_9_3 import MultiAgentArena
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import time

MAX_WIDTH = 10
MAX_HEIGHT = 10

ray.init(ignore_reinit_error=True)
                                                   
def env_creator(env_config):
    print(env_config)
    return MultiAgentArena(config=env_config, width=MAX_WIDTH, height=MAX_HEIGHT)  # return an env instance
register_env("multi_agent_arena", env_creator)

policies = { "policy1": PolicySpec(), "policy2": PolicySpec() }

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    if agent_id == "agent1":
        return "policy1"
    else:
        return "policy2"


config = (
    PPOConfig()
    .environment(env="multi_agent_arena")
    .framework("torch")
    .rollouts(create_env_on_local_worker=True)
    .debugging(seed=0,log_level="ERROR")
    .training(model={"fcnet_hiddens" : [64, 64]},lr=tune.grid_search([0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]))
    .multi_agent(
        policies=policies,
        policy_mapping_fn=policy_mapping_fn
    )
)


# ``Tuner.fit()`` allows setting a custom log directory (other than ``~/ray-results``)
tuner = ray.tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"episode_reward_mean": 10},
        checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True),
    ),
)

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(metric="episode_reward_mean", mode="max")

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint
